<a href="https://colab.research.google.com/github/cindyangelira/cindyangelira/blob/main/CatBoost%20Grid%20Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Import Package


import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
#Load the dataset


data = pd.read_csv('../input/creditcardfraud/creditcard.csv')

Before dive in more into dataset, it's always good to take a look at the data first.

In [ ]:
#Take a look at data


data.head()

Lets see the summary of dataset

In [ ]:
data.describe().T

In [ ]:
data.shape

In [ ]:
data.info()

Seem like the dataset doesn't contain any missing value.

In [ ]:
fraud = data[(data['Class'] ==1)]
normal = data[(data['Class'] != 1)]
labels = ['fraud','normal']

Generate a pie chart to show Fraud distribution.

In [ ]:
fig, ax = plt.subplots()
ax.pie([len(fraud),len(normal)], labels = labels, colors = ['blue','red'], autopct='%1.1f%%')
ax.set_title('Fraud CC Distribution')
plt.show()

We can see that the data is highly imbalanced. Lets take a closer look to the distribution of fraud cc

In [ ]:
fraud_pct = np.round(len(fraud)/len(data)*100, 4)
fraud_pct

Percentage of fraud cc in this dataset is 0.1727 percent.

When we look back to variable amount, this variable has large spread of value. So we scale the data as unscaled input variables can result in a slow or unstable learning process.

# DATA PREPROCESSING

In [1]:
from sklearn.preprocessing import StandardScaler


data['amount'] = StandardScaler().fit_transform(data['Amount'].values.reshape(-1,1))

NameError: ignored

**DROP VARIABLE**

In [ ]:
#Next we drop unnessesary variable (Time and Amount)


df = data.drop(['Time','Amount'], axis=1)

Lets look at our new dataframe

In [ ]:
df.head()

**DEFINE X AND Y**

In [ ]:
X = df.drop('Class', axis=1)
y = df['Class']

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
from imblearn.over_sampling import SMOTE


In [ ]:
smote = SMOTE(sampling_strategy ='minority')
X_sm, y_sm = smote.fit_resample(X, y)

In [ ]:
X_sm.shape

In [ ]:
y_sm.shape

In [ ]:
y_sm.head()

In [ ]:
y_sm==1

In [ ]:
fig, ax = plt.subplots()
ax.pie([len(y_sm==1),len(y_sm!=1)], labels = ['fraud', 'normal'], colors = ['blue','red'], autopct='%1.1f%%')
ax.set_title('Fraud CC Distribution')
plt.show()

**TRAIN TEST SPLIT**

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
#Set seed


seed=42
X_train, X_test, y_train, y_test = train_test_split(X_sm, y_sm, test_size = 0.25, random_state = seed, stratify = y_sm)

# CATBOOST CLASSIFIER

In [ ]:
#!pip install catboost

In [ ]:
import catboost

Import model solving classification task

In [ ]:
from catboost import CatBoostClassifier

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
cbc = CatBoostClassifier(loss_function= 'Logloss',eval_metric = 'AUC:hints=skip_train~false', random_seed = seed)

for further details of catboost.
https://catboost.ai/en/docs/concepts/python-reference_catboostclassifier

In [ ]:
params = {
    'depth': [6,8,10],
    'iterations': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.3],
    }

In [ ]:
%%time
grid = GridSearchCV(estimator=cbc, param_grid = params, cv = 3, n_jobs=-1)
grid.fit(X_train, y_train)

In [ ]:
grid.best_estimator_

In [ ]:
grid.best_score_

In [ ]:
grid.best_params_

In [ ]:
#grid_cbc.fit(X_train, y_train)

In [ ]:
#print(" Results from Grid Search " )
#print("\n The best estimator across ALL searched params:\n",grid_cbc.best_estimator_)
#print("\n The best score across ALL searched params:\n",grid_cbc.best_score_)
#print("\n The best parameters across ALL searched params:\n",grid_cbc.best_params_)